In [6]:
from langchain.chat_models import init_chat_model
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode, tools_condition

In [7]:
from dotenv import load_dotenv
load_dotenv()

True

In [8]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [9]:
@tool
def get_stock_price(symbol: str) -> float:
    '''Return the current price of a stock given the stock symbol
    :param symbol: stock symbol
    :return: current price of the stock
    '''
    return {
        "MSFT": 200.3,
        "AAPL": 100.4,
        "AMZN": 150.0,
        "RIL": 87.6
    }.get(symbol, 0.0)

tools = [get_stock_price]

# llm = init_chat_model("google_genai:gemini-2.0-flash")
llm = init_chat_model(model="llama3.1:8b", model_provider="ollama")
llm_with_tools = llm.bind_tools(tools)

In [10]:
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

builder = StateGraph(State)

builder.add_node(chatbot)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "chatbot")
builder.add_conditional_edges("chatbot", tools_condition)
builder.add_edge("tools", "chatbot")
graph = builder.compile()

# from IPython.display import Image, display
# display(Image(graph.get_graph().draw_mermaid_png()))

In [11]:
state = graph.invoke({"messages": [{"role": "user", "content": "What is the price of AAPL stock right now?"}]})
print(state["messages"][-1].content)

The current price of AAPL (Apple Inc.) stock is $100.4.


In [12]:
state = graph.invoke({"messages": [{"role": "user", "content": "Who invented theory of relativity? print person name only"}]})
print(state["messages"][-1].content)

Albert


In [13]:
msg = "I want to buy 20 AMZN stocks using current price. Then 15 MSFT. What will be the total cost?"

state = graph.invoke({"messages": [{"role": "user", "content": msg}]})
print(state["messages"][-1].content)

To find the total cost, we need to use the get_stock_price tool to get the current prices for AMZN and MSFT, then multiply those prices by the quantities you want to buy.

The current price for AMZN is $100.0.
The current price for MSFT is $150.0.

The total cost of 20 AMZN stocks would be: 20 * $100.0 = $2000.00
The total cost of 15 MSFT stocks would be: 15 * $150.0 = $2250.00

So the total cost for both purchases is: $2000.00 + $2250.00 = $4250.00
